# Rakenna luokittelumalli: Herkulliset aasialaiset ja intialaiset keittiöt


## Keittiöluokittelijat 2

Tässä toisessa luokittelutunnissa tutkimme `lisätapoja` kategorisen datan luokitteluun. Opimme myös, mitä seurauksia on, kun valitsemme yhden luokittelijan toisen sijaan.

### [**Esiluennon kysely**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Edellytykset**

Oletamme, että olet suorittanut aiemmat oppitunnit, sillä jatkamme joitakin aiemmin opittuja käsitteitä.

Tätä oppituntia varten tarvitsemme seuraavat paketit:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) on [R-pakettien kokoelma](https://www.tidyverse.org/packages), joka tekee datatieteestä nopeampaa, helpompaa ja hauskempaa!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) -kehys on [pakettien kokoelma](https://www.tidymodels.org/packages/) mallinnukseen ja koneoppimiseen.

-   `themis`: [themis-paketti](https://themis.tidymodels.org/) tarjoaa lisäreseptivaiheita epätasapainoisen datan käsittelyyn.

Voit asentaa ne seuraavasti:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Vaihtoehtoisesti alla oleva skripti tarkistaa, onko sinulla tarvittavat paketit tämän moduulin suorittamiseen, ja asentaa ne puolestasi, jos ne puuttuvat.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Luokittelukartta**

Edellisessä [oppitunnissamme](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) yritimme vastata kysymykseen: kuinka valitsemme useiden mallien välillä? Suurelta osin se riippuu datan ominaisuuksista ja ongelman tyypistä, jonka haluamme ratkaista (esimerkiksi luokittelu tai regressio).

Aiemmin opimme eri vaihtoehdoista, joita sinulla on datan luokittelussa Microsoftin huijauslistan avulla. Pythonin koneoppimisen kehys, Scikit-learn, tarjoaa vastaavan mutta tarkemman huijauslistan, joka voi auttaa kaventamaan valintaa estimointien (toinen termi luokittelijoille) välillä:

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Vinkki: [katso tämä kartta verkossa](https://scikit-learn.org/stable/tutorial/machine_learning_map/) ja klikkaa polkua pitkin lukeaksesi dokumentaatiota.  
>  
> [Tidymodels-viitesivusto](https://www.tidymodels.org/find/parsnip/#models) tarjoaa myös erinomaista dokumentaatiota eri mallityypeistä.

### **Suunnitelma** 🗺️

Tämä kartta on erittäin hyödyllinen, kun sinulla on selkeä käsitys datastasi, sillä voit "kulkea" sen polkuja pitkin päätökseen:

-   Meillä on \>50 näytettä

-   Haluamme ennustaa kategorian

-   Meillä on merkittyä dataa

-   Meillä on alle 100 000 näytettä

-   ✨ Voimme valita Linear SVC:n

-   Jos se ei toimi, koska meillä on numeerista dataa

    -   Voimme kokeilla ✨ KNeighbors Classifieria

        -   Jos sekään ei toimi, kokeile ✨ SVC:tä ja ✨ Ensemble Classifiers -malleja

Tämä on erittäin hyödyllinen polku seurattavaksi. Nyt sukelletaan suoraan asiaan käyttämällä [tidymodels](https://www.tidymodels.org/) -mallinnuskehystä: johdonmukainen ja joustava kokoelma R-paketteja, jotka on kehitetty edistämään hyviä tilastollisia käytäntöjä 😊.

## 2. Jaa data ja käsittele epätasapainoista aineistoa.

Aiemmista oppitunneistamme opimme, että keittiöissämme oli joukko yhteisiä ainesosia. Lisäksi keittiöiden määrässä oli melko epätasainen jakauma.

Käsittelemme nämä seuraavasti:

-   Poistamme yleisimmät ainesosat, jotka aiheuttavat sekaannusta eri keittiöiden välillä, käyttämällä `dplyr::select()`-funktiota.

-   Käytämme `recipe`-objektia, joka esikäsittelee datan mallinnusta varten soveltamalla `over-sampling`-algoritmia.

Kävimme yllä olevan jo läpi edellisessä oppitunnissa, joten tämän pitäisi olla helppoa 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Käsittele epätasapainoista dataa

Epätasapainoinen data vaikuttaa usein negatiivisesti mallin suorituskykyyn. Monet mallit toimivat parhaiten, kun havaintojen määrä on tasainen, ja siksi ne voivat kohdata haasteita epätasapainoisen datan kanssa.

Epätasapainoisten datajoukkojen käsittelyyn on pääasiassa kaksi tapaa:

-   lisätä havaintoja vähemmistöluokkaan: `Yliotanta` esimerkiksi käyttämällä SMOTE-algoritmia, joka synteettisesti luo uusia vähemmistöluokan esimerkkejä näiden tapausten lähimpien naapureiden avulla.

-   poistaa havaintoja enemmistöluokasta: `Aliotanta`

Edellisessä oppitunnissamme näytimme, kuinka epätasapainoisia datajoukkoja voidaan käsitellä käyttämällä `reseptiä`. Reseptiä voidaan ajatella suunnitelmana, joka kuvaa, mitä vaiheita datajoukolle tulisi soveltaa, jotta se olisi valmis data-analyysiin. Meidän tapauksessamme haluamme saada tasaisen jakauman keittiötyyppien määrässä `opetusjoukkoamme` varten. Mennään suoraan asiaan.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Nyt olemme valmiita kouluttamaan malleja 👩‍💻👨‍💻!

## 3. Monitermi-regressiomallien ulkopuolella

Edellisessä oppitunnissa tarkastelimme monitermi-regressiomalleja. Tutkitaan nyt joustavampia luokittelumalleja.

### Tukivektorikoneet

Luokittelun yhteydessä `Tukivektorikoneet` ovat koneoppimistekniikka, joka pyrkii löytämään *hyper-tason*, joka "parhaiten" erottaa luokat toisistaan. Katsotaanpa yksinkertaista esimerkkiä:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ ei erota luokkia. H2~ erottaa, mutta vain pienellä marginaalilla. H3~ erottaa ne suurimmalla mahdollisella marginaalilla.

#### Lineaarinen tukivektoriluokitin

Tukivektoriklusterointi (SVC) kuuluu tukivektoriautomaattien (Support-Vector Machines) koneoppimistekniikoiden perheeseen. SVC:ssä hypertaso valitaan siten, että se erottaa `suurimman osan` harjoitusaineiston havainnoista oikein, mutta `saattaa luokitella väärin` joitakin havaintoja. Sallimalla joidenkin pisteiden olla väärällä puolella, SVM:stä tulee kestävämpi poikkeamille ja se yleistyy paremmin uuteen dataan. Parametri, joka säätelee tätä poikkeamaa, tunnetaan nimellä `cost`, jonka oletusarvo on 1 (katso `help("svm_poly")`).

Luodaan lineaarinen SVC asettamalla `degree = 1` polynomiseen SVM-malliin.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Nyt kun olemme tallentaneet esikäsittelyvaiheet ja mallin määrittelyn *työnkulkuun*, voimme jatkaa lineaarisen SVC:n kouluttamista ja samalla arvioida tuloksia. Suorituskykymittareita varten luodaan mittaristo, joka arvioi: `accuracy` (tarkkuus), `sensitivity` (herkkyys), `Positive Predicted Value` (positiivinen ennustearvo) ja `F Measure` (F-mittari).

> `augment()` lisää sarakkeen/sarakkeita ennusteille annettuun dataan.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Tukivektorikone

Tukivektorikone (SVM) on tukivektoriluokittelijan laajennus, joka mahdollistaa epälineaarisen rajan luokkien välillä. Pohjimmiltaan SVM:t käyttävät *ydintemppua* laajentaakseen piirretilaa, jotta ne voivat mukautua epälineaarisiin suhteisiin luokkien välillä. Yksi suosittu ja erittäin joustava ydinfunktio, jota SVM:t käyttävät, on *radiaalinen basisfunktio.* Katsotaan, miten se suoriutuu datassamme.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Paljon parempi 🤩!

> ✅ Katso:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> lisälukemista varten.

### Lähimmän naapurin luokittelijat

*K*-lähimmän naapurin (KNN) algoritmi ennustaa jokaisen havainnon sen *samankaltaisuuden* perusteella muihin havaintoihin.

Sovitetaan yksi algoritmi dataamme.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Näyttää siltä, että tämä malli ei suoriudu kovin hyvin. Todennäköisesti mallin argumenttien muuttaminen (katso `help("nearest_neighbor")`) parantaa mallin suorituskykyä. Muista kokeilla tätä.

> ✅ Katso:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> saadaksesi lisätietoa *K*-lähimpien naapureiden luokittelijoista.

### Yhdistelmäluokittelijat

Yhdistelmäalgoritmit toimivat yhdistämällä useita perusestimaattoreita optimaalisen mallin tuottamiseksi joko:

`bagging`: käyttämällä *keskiarvoistavaa funktiota* kokoelmaan perusmalleja

`boosting`: rakentamalla mallisarjan, jossa mallit parantavat toistensa ennustustarkkuutta.

Aloitetaan kokeilemalla Random Forest -mallia, joka rakentaa suuren joukon päätöspuita ja käyttää keskiarvoistavaa funktiota paremman kokonaismallin luomiseksi.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Hyvää työtä 👏!

Kokeillaan myös Boosted Tree -mallia.

Boosted Tree määrittelee ensemble-menetelmän, joka luo sarjan peräkkäisiä päätöspuita, joissa jokainen puu perustuu edellisten puiden tuloksiin pyrkien vähentämään virhettä asteittain. Se keskittyy väärin luokiteltujen kohteiden painoihin ja säätää seuraavan luokittelijan sovitusta korjatakseen virheet.

Tämän mallin sovittamiseen on erilaisia tapoja (katso `help("boost_tree")`). Tässä esimerkissä sovitamme Boosted Tree -mallin `xgboost`-moottorin avulla.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Katso:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Tutkii AdaBoost-mallia, joka on hyvä vaihtoehto xgboostille.
>
> oppiaksesi lisää Ensemble-luokittelijoista.

## 4. Lisäosa - useiden mallien vertailu

Olemme sovittaneet melko monta mallia tässä laboratoriossa 🙌. Useiden työnkulkujen luominen eri esikäsittelyjen ja/tai mallimääritysten pohjalta ja suorituskykymittareiden laskeminen yksi kerrallaan voi käydä työlääksi.

Katsotaan, voimmeko ratkaista tämän luomalla funktion, joka sovittaa listan työnkulkuja harjoitusaineistoon ja palauttaa suorituskykymittarit testiaineiston perusteella. Käytämme `map()` ja `map_dfr()` -funktioita [purrr](https://purrr.tidyverse.org/) -paketista soveltaaksemme funktioita listan jokaiseen elementtiin.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) -funktiot mahdollistavat monien for-silmukoiden korvaamisen koodilla, joka on sekä tiiviimpää että helpommin luettavaa. Paras paikka oppia [`map()`](https://purrr.tidyverse.org/reference/map.html) -funktioista on [iteraatio-luku](http://r4ds.had.co.nz/iteration.html) kirjassa R for Data Science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) -paketti mahdollistaa käyttäjille suuren määrän mallien luomisen ja helpon sovittamisen, mutta se on pääasiassa suunniteltu toimimaan uudelleennäytteistystekniikoiden, kuten `ristivahvistuksen`, kanssa. Tätä lähestymistapaa emme ole vielä käsitelleet.

## **🚀Haaste**

Jokaisella näistä tekniikoista on suuri määrä parametreja, joita voit säätää, esimerkiksi `cost` SVM:ssä, `neighbors` KNN:ssä, `mtry` (satunnaisesti valitut ennustajat) Random Forestissa.

Tutki kunkin oletusparametreja ja pohdi, mitä näiden parametrien säätäminen tarkoittaisi mallin laadulle.

Lisätietoja tietystä mallista ja sen parametreista saat käyttämällä: `help("model")`, esim. `help("rand_forest")`.

> Käytännössä *arvioimme* yleensä *parhaat arvot* näille kouluttamalla useita malleja `simuloidulla aineistolla` ja mittaamalla, kuinka hyvin nämä mallit suoriutuvat. Tätä prosessia kutsutaan **virittämiseksi**.

### [**Luennon jälkeinen kysely**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Kertaus & Itseopiskelu**

Näissä oppitunneissa on paljon ammattikieltä, joten käytä hetki aikaa tutustuaksesi [tähän sanastoon](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott), joka sisältää hyödyllisiä termejä!

#### KIITOS:

[`Allison Horst`](https://twitter.com/allison_horst/) upeista kuvituksista, jotka tekevät R:stä kutsuvamman ja innostavamman. Löydät lisää kuvituksia hänen [galleriastaan](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) ja [Jen Looper](https://www.twitter.com/jenlooper) alkuperäisen Python-version luomisesta ♥️

Mukavia oppimishetkiä,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Taide: @allison_horst</figcaption>



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Pyrimme tarkkuuteen, mutta huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulee pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskääntämistä. Emme ole vastuussa tämän käännöksen käytöstä aiheutuvista väärinkäsityksistä tai virhetulkinnoista.
